In [1]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# URL del archivo en el bucket público de AWS
url = "https://hybridge-education-machine-learning-datasets.s3.us-east-1.amazonaws.com/Fraud.csv"
df = pd.read_csv(url)

In [3]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [4]:
len(df)

6362620

# ¡¡¡6,362,620 OBSERVACIONES!!!

Bloque 1: Limpieza y Codificación de Features  ¡¡¡6,362,620 OBSERVACIONES!!!

In [5]:
# Definición de la variable objetivo
TARGET_COL = 'isFraud'

# 1. Codificar 'type' (es nominal) y crear un dataframe temporal
df_model = pd.get_dummies(df.copy(), columns=['type'], drop_first=True)

# 2. Eliminar columnas no necesarias para el modelo:
# Identificadores (alta cardinalidad) y la bandera humana
columns_to_drop = ['nameOrig', 'nameDest', 'isFlaggedFraud']

# Usamos errors='ignore' por si alguna columna ya fue eliminada o renombrada
df_model = df_model.drop(columns=columns_to_drop, errors='ignore')

print("DataFrame listo para modelar con One-Hot Encoding:")
print(df_model.shape)

DataFrame listo para modelar con One-Hot Encoding:
(6362620, 11)


Bloque 2: Split, Escala, Entrenamiento y Evaluación (Captura Final)

In [6]:
# 1. Definir X e y con el dataframe limpio y codificado
X = df_model.drop(TARGET_COL, axis=1)
y = df_model[TARGET_COL]

# 2. División de datos
# Usamos test_size=0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 3. Escalar las variables (CRUCIAL para Regresión Logística)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# 4. Entrenar el modelo
# Usamos 'saga' y n_jobs=-1 para manejar el gran volumen de datos (3.3M)
model = LogisticRegression(solver='saga', random_state=42, n_jobs=-1, max_iter=200) 
model.fit(X_train_scaled, y_train)

# 5. Predecir y mostrar la evaluación final
y_pred = model.predict(X_test_scaled)

print("=== EVALUACIÓN FINAL DEL MODELO DE REGRESIÓN LOGÍSTICA ===")
print(classification_report(y_test, y_pred))
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred))

/Users/kurisu/Desktop/ML/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


=== EVALUACIÓN FINAL DEL MODELO DE REGRESIÓN LOGÍSTICA ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1906351
           1       0.99      0.23      0.37      2435

    accuracy                           1.00   1908786
   macro avg       1.00      0.61      0.69   1908786
weighted avg       1.00      1.00      1.00   1908786


Matriz de Confusión:
[[1906346       5]
 [   1878     557]]
